In [1]:
import pandas as pd; import numpy as np; import janitor as jn
pd.__version__

'2.2.1'

In [2]:
# events = pd.read_csv('/Users/samuel.oranyeli/Downloads/results.csv', parse_dates=['start','end']).iloc[:, 1:]

In [3]:
# a = (events
# .conditional_join(
#     events,
#     ('start', 'end', '<='),
#     ('end', 'start', '>='),
#     # ('id', 'id', '!='),
#     # ('audience','audience','>'),
#     use_numba = True,
#     df_columns = ['id', 'start', 'end'],
#     right_columns = ['id', 'start', 'end'])
# )

# a

In [2]:
C=dict(key=[f"c{num}" for num in range(1,8)],
       vol=[35,15,5,35,18,90,17],
       profit=[45,35,55,12,15,55,11],
       unitsSold=[15,10,30,10,15,80,2],
       keyy = range(1,8)
       )
C=pd.DataFrame(C)


D=dict(key=[f'd{num}' for num in range(1,9)],
       vol=[20,50,15,16,40,20,40,2],
       profit=[30,10,12,52,35,20,30,57],
       unitsSold=[20,35,10,12,40,30,5,15],
       keyy=range(8,0,-1)
       )
D=pd.DataFrame(D)

In [5]:
D.index

RangeIndex(start=0, stop=8, step=1)

In [3]:
(C
.conditional_join(
    D, 
    
    ('profit','profit','>='),
    # ('keyy','keyy','=='),
  # ('unitsSold','unitsSold','>='),
  ('vol','vol','<='),
  # ('vol','profit','>'),
    use_numba=True)
)

left                           right                          
    key vol profit unitsSold keyy   key vol profit unitsSold keyy
0    c3   5     55        30    3    d3  15     12        10    6
1    c3   5     55        30    3    d4  16     52        12    5
2    c3   5     55        30    3    d1  20     30        20    8
3    c3   5     55        30    3    d6  20     20        30    3
4    c3   5     55        30    3    d2  50     10        35    7
5    c3   5     55        30    3    d5  40     35        40    4
6    c3   5     55        30    3    d7  40     30         5    2
7    c1  35     45        15    1    d2  50     10        35    7
8    c1  35     45        15    1    d5  40     35        40    4
9    c1  35     45        15    1    d7  40     30         5    2
10   c2  15     35        10    2    d3  15     12        10    6
11   c2  15     35        10    2    d1  20     30        20    8
12   c2  15     35        10    2    d6  20     20        30    3
13   c2  15     35        10    2    d2  50     10        35    7
14   c2  15     35        10    2    d5  40     35        40    4
15   c2  15     35        10    2    d7  40     30         5    2
16   c4  35     12        10    4    d2  50     10        35    7
17   c5  18     15        15    5    d2  50     10        35    7
18   c7  17     11         2    7    d2  50     10        35    7

In [4]:
(C
.conditional_join(
    D, 
    
    ('profit','profit','>='),
    # ('keyy','keyy','=='),
  # ('unitsSold','unitsSold','>='),
  ('vol','vol','<='),
  # ('vol','profit','>'),
    use_numba=False)
)

left                           right                          
    key vol profit unitsSold keyy   key vol profit unitsSold keyy
0    c1  35     45        15    1    d2  50     10        35    7
1    c1  35     45        15    1    d7  40     30         5    2
2    c1  35     45        15    1    d5  40     35        40    4
3    c2  15     35        10    2    d2  50     10        35    7
4    c2  15     35        10    2    d3  15     12        10    6
5    c2  15     35        10    2    d6  20     20        30    3
6    c2  15     35        10    2    d1  20     30        20    8
7    c2  15     35        10    2    d7  40     30         5    2
8    c2  15     35        10    2    d5  40     35        40    4
9    c3   5     55        30    3    d2  50     10        35    7
10   c3   5     55        30    3    d3  15     12        10    6
11   c3   5     55        30    3    d6  20     20        30    3
12   c3   5     55        30    3    d1  20     30        20    8
13   c3   5     55        30    3    d7  40     30         5    2
14   c3   5     55        30    3    d5  40     35        40    4
15   c3   5     55        30    3    d4  16     52        12    5
16   c4  35     12        10    4    d2  50     10        35    7
17   c5  18     15        15    5    d2  50     10        35    7
18   c7  17     11         2    7    d2  50     10        35    7

In [5]:
from string import ascii_lowercase
np.random.seed(1)
n = 20; k = 20
n = 2_000_000; k = 2_000
mapp = {k:v for v,k in enumerate(ascii_lowercase)}
idx1 = np.random.randint(0, high = 2_00, size = n)
idx2 = np.random.randint(0, high = 3_00, size = n)

d1 = dict(x = np.random.choice(list(ascii_lowercase[:5]), size=n),
          start = np.minimum(idx1, idx2),
          end = np.maximum(idx1, idx2),
          )


d2 = dict(x = np.random.choice(list(ascii_lowercase[:15]), size=k),
          pos1 = np.random.randint(low=60, high = 151, size=k))

d1 = pd.DataFrame(d1)
d2 = pd.DataFrame(d2)
d1 = d1.assign(xx=lambda df: df.x.map(mapp))
d2 = d2.assign(xx=lambda df: df.x.map(mapp))

In [6]:
bla = (d2
.conditional_join(
    d1, 
    ('pos1', 'start', '>'), 
     ('pos1', 'end', '<'), 
   # ('xx', 'xx', '=='), 
    use_numba=True)
)

bla

left          right              
              x pos1  xx     x start  end xx
0             l  150  11     e    59  168  4
1             l  150  11     b    59  266  1
2             l  150  11     a    59  164  0
3             l  150  11     b    59  282  1
4             l  150  11     b    59  224  1
...         ...  ...  ..   ...   ...  ... ..
1924132123    g   60   6     c     0  156  2
1924132124    g   60   6     e     0  262  4
1924132125    g   60   6     d     0  258  3
1924132126    g   60   6     e     0  173  4
1924132127    g   60   6     a     0  265  0

[1924132128 rows x 7 columns]

In [10]:
%%timeit
(d2
.conditional_join(
    d1, 
    ('pos1', 'start', '>'), 
     ('pos1', 'end', '<'), 
   # ('xx', 'xx', '=='), 
    use_numba=True)
)

to the enddddd
to the enddddd
to the enddddd
to the enddddd


KeyboardInterrupt: 

In [ ]:
%%timeit
(d2
.conditional_join(
    d1, 
    ('pos1', 'start', '>'), 
     ('pos1', 'end', '<'), 
   # ('xx', 'xx', '=='), 
    use_numba=False)
)

2min 3s ± 4.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# %timeit d1.sort_values('start')